# KoBERT finetuning

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install ipywidgets  # for vscode
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-p0o2s6si
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-p0o2s6si


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model
from sklearn.model_selection import train_test_split
                                                         


In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
## CPU
# device = torch.device("cpu")

## GPU
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
!wget -O .cache/ratings_train.txt http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_train.txt
!wget -O .cache/ratings_test.txt http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_test.txt

--2022-03-30 13:21:54--  http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_train.txt
Resolving skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)... 52.219.60.54
Connecting to skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)|52.219.60.54|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘.cache/ratings_train.txt’

.cache/ratings_trai 100%[===================>]  13.95M  7.62MB/s    in 1.8s    

2022-03-30 13:21:56 (7.62 MB/s) - ‘.cache/ratings_train.txt’ saved [14628807/14628807]

--2022-03-30 13:21:56--  http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_test.txt
Resolving skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)... 52.219.60.54
Connecting to skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)|52.219.60.54|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4893335 (4.7M

In [ ]:
dataset = nlp.data.TSVDataset("/content/drive/MyDrive/캡디/한국어_단발성_대화_데이터셋.xlsx - Sheet1.tsv", field_indices=[0,1], num_discard_samples=1)
# dataset_test = nlp.data.TSVDataset(".cache/ratings_test.txt", field_indices=[1,2], num_discard_samples=1)

In [ ]:
tmp_dataset_train = np.array(dataset)
# print(dataset_train[0])
print(tmp_dataset_train.shape)
dataset_train = []
dataset_test = []
p = 0.9
# print(len(dataset[:,1]))
train_idx = []
test_idx = []

# for i in range(7):
#   tmp_idx = np.where(tmp_dataset_train[:,1] == str(i))[0]
#   tmp_train_idx = int(len(tmp_idx)*p)

#   # np.concatenate((dataset_train, tmp_dataset_train[tmp_idx[:tmp_train_idx]]), axis=0)
#   # np.concatenate((dataset_test, tmp_dataset_train[tmp_idx[tmp_train_idx:]]), axis=0)
#   dataset_train += tmp_dataset_train[tmp_idx[:tmp_train_idx]].tolist()
#   dataset_test += tmp_dataset_train[tmp_idx[tmp_train_idx:]].tolist()

dataset_train, dataset_test = train_test_split(dataset, test_size=0.1, stratify=tmp_dataset_train[:,1])
# dataset_train, dataset_test = train_test_split(dataset, test_size=0.2, stratify=tmp_dataset_train[:,1])

print(len(dataset_train), len(dataset_test))

(38594, 2)
30875 7719


In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [ ]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 30
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        # nn.init.xavier_uniform_(self.classifier.weight, 0.0)
        if dr_rate is not None:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        out = pooler
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.bert.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.bert.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
    {'params': [p for n, p in model.classifier.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.classifier.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# optimizer_grouped_parameters = [
#     {'params': [p for n, p in model.classifier.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01, 'lr' : 0.001},
#     {'params': [p for n, p in model.classifier.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0, 'lr' : 0.001}
# ]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
#official code drop_rate : 0.1
for i in range(12):
  model.bert.encoder.layer[i].attention.self.dropout = nn.Dropout(0.5)
  model.bert.encoder.layer[i].attention.output.dropout = nn.Dropout(0.5)
  model.bert.encoder.layer[i].output.dropout = nn.Dropout(0.5)


In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/483 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.9362025260925293 train acc 0.1875
epoch 1 batch id 201 loss 1.9759420156478882 train acc 0.14769900497512436
epoch 1 batch id 401 loss 1.9329965114593506 train acc 0.14974283042394015
epoch 1 train acc 0.15108599800628786


  0%|          | 0/121 [00:00<?, ?it/s]

epoch 1 test acc 0.16942810976901884


...

epoch 22 batch id 1 loss 0.6937263011932373 train acc 0.78125
epoch 22 batch id 201 loss 0.6472786664962769 train acc 0.7151741293532339
epoch 22 batch id 401 loss 0.7580546140670776 train acc 0.7124376558603491
epoch 22 train acc 0.7112903535004985


  0%|          | 0/121 [00:00<?, ?it/s]

epoch 22 test acc 0.4824347054460691


  0%|          | 0/483 [00:00<?, ?it/s]

epoch 23 batch id 1 loss 0.6927011013031006 train acc 0.734375
epoch 23 batch id 201 loss 0.6673315167427063 train acc 0.716806592039801
epoch 23 batch id 401 loss 0.7236607074737549 train acc 0.7167238154613467


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/캡디/drop_05.pth')